In [7]:
from openai import OpenAI
from pydantic import BaseModel,Field
from duckduckgo_search import DDGS
from IPython.display import Markdown
import json
from langchain_community.tools import DuckDuckGoSearchRun
import os
import re
import requests
import datetime
from langchain_google_community import GoogleSearchAPIWrapper

In [8]:
#initialize the agent with the api key
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

Model = "gpt-4o-mini"

In [9]:
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [10]:
def search_for_topic(topic: str) -> str:
    """
    Searches for a given topic using DuckDuckGo and returns the search results.

    Args:
        topic (str): The topic to search for.

    Returns:
        str: The search results if successful, otherwise an error message.
    """
    try:
        # search = DuckDuckGoSearchRun()
        # return search.invoke(topic)
        print(topic)
        search = GoogleSearchAPIWrapper()
        return search.run(topic)
    except Exception as e:
        return f"An error occurred while searching: {str(e)}"

In [11]:
POST_CREATION_PROMPT = """
You are a seasoned LinkedIn content strategist known for crafting high-engagement posts that generate meaningful professional conversations. Create a compelling LinkedIn post about "{topic}" based on the provided context.

Context: {context}

Your post should:
1. Open with a powerful hook that captures attention in the first 3 lines (visible before "see more" appears)
2. Include one specific data point, personal insight, or contrarian viewpoint that challenges conventional thinking
3. Structure content with strategic line breaks and spacing for mobile readability
4. End with a clear call-to-action that invites thoughtful responses
5. Include 3-5 relevant hashtags that balance popularity and specificity
6. Maintain an authentic, conversational tone while demonstrating expertise
7. Keep the total length between 1000-1300 characters for optimal engagement
8. Do not use markdown formatting in your response
9. Include 2-4 relevant emojis strategically placed to enhance key points (not just decorative)

Aim for a post that positions the author as a thoughtful industry voice rather than just sharing information. Provide the post in plain text format exactly as it would appear on LinkedIn."""

In [12]:
def create_post(context: str, topic: str) -> str:
    """
    Generates a post based on the provided context and topic using a chat completion model.

    Args:
        context (str): The content or context to include in the post.
        topic (str): The topic around which the post is created.

    Returns:
        str: The generated post formatted with ANSI markdown.
    """
    prompt = POST_CREATION_PROMPT.format(context=context, topic=topic)
    response = client.chat.completions.create(
        model=Model,
        messages=[{"role": "user", "content": prompt}],
    )
    post_content = response.choices[0].message.content
    return post_content

In [13]:
import os
import requests
import logging

def post_linkedin_content(content: str) -> str:
    """
    Posts the provided content to LinkedIn using the LinkedIn UGC API.

    This function sends a POST request to LinkedIn's API to create a new public post
    authored by a predefined LinkedIn user. It uses a hard-coded LinkedIn person ID
    and an access token (retrieved from an environment variable) for authentication.
    If the post is successfully created (HTTP 201), a success message is returned.
    Otherwise, an error message is returned detailing the status code and error response.

    Args:
        content (str): The text content to be posted on LinkedIn.

    Returns:
        str: A message indicating the success or failure of the post operation.
    """
    # Hard-coded credentials (consider using secure storage in production)
    access_token = os.getenv("LINKEDIN_ACCESS_TOKEN")
    if not access_token:
        logging.error("Access token not found. Please set the LINKEDIN_ACCESS_TOKEN environment variable.")
        return "Missing access token."

    post_url = "https://api.linkedin.com/v2/ugcPosts"
    userinfo_url = "https://api.linkedin.com/v2/userinfo"

    # Headers for the GET request to fetch user info
    headers_get = {
        "Authorization": f"Bearer {access_token}"
    }

    try:
        user_response = requests.get(userinfo_url, headers=headers_get)
        user_response.raise_for_status()
        user_info = user_response.json()
        # Assuming the 'sub' field contains the LinkedIn person ID
        person_id = user_info.get('sub')
        if not person_id:
            logging.error("User ID not found in the user info response.")
            return "User ID missing in response."
        logging.info(f"Fetched user info successfully. User ID: {person_id}")
    except requests.RequestException as err:
        logging.error("Error fetching user info from LinkedIn", exc_info=True)
        return f"Error fetching user info: {err}"

    # Headers for the POST request
    headers_post = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "X-Restli-Protocol-Version": "2.0.0"
    }

    post_body = {
        "author": f"urn:li:person:{person_id}",
        "lifecycleState": "PUBLISHED",
        "specificContent": {
            "com.linkedin.ugc.ShareContent": {
                "shareCommentary": {"text": content},
                "shareMediaCategory": "NONE"
            }
        },
        "visibility": {
            "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
        }
    }

    try:
        post_response = requests.post(post_url, headers=headers_post, json=post_body)
        post_response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.RequestException as err:
        logging.error("Failed to post content to LinkedIn", exc_info=True)
        return f"Failed to post content: {err}"

    if post_response.status_code == 201:
        logging.info("LinkedIn post created successfully.")
        return "Post successfully created!"
    else:
        error_message = f"Failed to post content: {post_response.status_code}\n{post_response.text}"
        logging.error(error_message)
        return error_message


In [14]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_for_topic",
            "description": "Retrieve the latest news and updates related to a specific topic.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {"type": "string", "description": "A keyword or subject for which to fetch news and current events."}
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_post",
            "description": "Generate a unique, plagiarism-free post using the provided context and topic.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {"type": "string", "description": "Background information to guide the post creation."},
                    "topic": {"type": "string", "description": "The central subject of the post content."}
                },
                "required": ["context", "topic"],
                "additionalProperties": False,
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "post_linkedin_content",
            "description": "Publish content directly to a LinkedIn profile.",
            "strict": True,
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {"type": "string", "description": "The content to be shared on LinkedIn."}
                },
                "required": ["content"],
                "additionalProperties": False
            }
        }
    }
]


# Dictionary mapping function names to their implementations
tool_implementations = {
    "search_for_topic": search_for_topic,
    "create_post": create_post, 
    "post_linkedin_content": post_linkedin_content
}

In [15]:
# code for executing the tools returned in the model's response
def handle_tool_calls(tool_calls, messages):
    
    for tool_call in tool_calls:   
        function = tool_implementations[tool_call.function.name]
        function_args = json.loads(tool_call.function.arguments)
        result = function(**function_args)
        messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
        
    return messages

In [16]:
SYSTEM_PROMPT = """
You are a helpful assistant that can Search lastest news on topics to create plagarism free linkedin content to post on LinkedIn.
"""

In [17]:
def run_agent(messages):
    print("Running agent with messages:", messages)

    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": SYSTEM_PROMPT}
            messages.append(system_prompt)

    while True:
        print("Making router call to OpenAI")
        response = client.chat.completions.create(
            model=Model,
            messages=messages,
            tools=tools,
        )
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls
        print("Received response with tool calls:", bool(tool_calls))

        # if the model decides to call function(s), call handle_tool_calls
        if tool_calls:
            print("Processing tool calls")
            messages = handle_tool_calls(tool_calls, messages)
        else:
            print("No tool calls, returning final response")
            return response.choices[0].message.content

In [18]:
result = run_agent('make a Post on top trending topics today')

Running agent with messages: make a Post on top trending topics today
Making router call to OpenAI
Received response with tool calls: True
Processing tool calls
trending topics
Making router call to OpenAI
Received response with tool calls: True
Processing tool calls
Making router call to OpenAI
Received response with tool calls: True
Processing tool calls
Making router call to OpenAI
Received response with tool calls: False
No tool calls, returning final response
